In [1]:
# 装饰器基础，装饰器就是一个参数为函数的函数
# 可以处理并返回作为参数传入的函数，或者替换为另一个函数或可调用对象
# @decorate
# def target():
#   print("running target()")
  
# def target():
#   print("running target()")
  
# target = decorate(target)
# 以上2中写法的效果一摸一样，装饰器其实只是一个语法糖

def deco(func):
  def inner():
    print('running inner()')
  return inner

@deco
def target():
  print('running target()')
  
target()
print(target) # target是inner的引用

running inner()
<function deco.<locals>.inner at 0x107a96f20>


In [4]:
# 装饰器会在被装饰的函数定义之后立即执行，通常是在模块导入时
registry = []

def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1 ():
  print('running f1()')

@register
def f2():
  print('running f2()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()

main()# 将会先运行装饰器函数
# 如果将这段代码封装到一个模块中，在import这个模块时，会立马运行装饰器函数

running register(<function f1 at 0x107a2bec0>)
running register(<function f2 at 0x1079ffce0>)
running main()
registry -> [<function f1 at 0x107a2bec0>, <function f2 at 0x1079ffce0>]
running f1()
running f2()


In [ ]:
# 使用装饰器改进策略模式
promos = []

def promotion(promo_func):
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity(order):
  return order.total()*0.05 if order.customer.fidelity >=1000 else 0

@promotion
def bulk_item(order):
  discount = 0 
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total()*0.1
  return discount

def best_promo():
  return max([promo(order) for promo in promos])

In [7]:
# 变量作用域规则
# 函数会优先在局部寻找变量，找不到时候再找全局变量
b = 6
def f1(a):
  print(a)
  print(b)
  
def f2(a):
  print(a)
  print(b) # 这里会报错b没有赋值
  b = 1

def f3(a):
  global b # 用global让函数去找全局而不是局部的变量b
  print(a)
  print(b)
  b = 1  

f1(3)
f3(3)
f2(3)

3
6
3
6


In [12]:
# 注意，只有嵌套函数时，才会有闭包的问题。
# 闭包，指延伸了作用域的函数，其中包含了在函数中引用，但是不在定义体中定义的非全局变量
# 闭包，能访问定义体之外定义的非全局变量

# 用类实现avg函数
class AvgFunc():
  def __init__(self):
    self.series = []
    
  def __call__(self, new_val):
    self.series.append(new_val)
    total = sum(self.series)
    return total/len(self.series)

avg = AvgFunc()
print(avg(10))
print(avg(11))
print(avg(12))

# 用闭包函数实现avg
def avg_wrapper():
  # series是一个自由变量，未在本地作用域绑定的变量
  # series 的绑定在返回的 avg 函数的 __closure__ 属性中。avg.__closure__ 中的各个元
  # 素对应于 avg.__code__.co_freevars 中的一个名称。这些元素是 cell 对象，有个 cell_
  # contents 属性，保存着真正的值。
  series = []
   
  def avg(num):
    series.append(num)
    total = sum(series)
    return total/len(series)
  return avg

avg = avg_wrapper()
print(avg(10))
# 闭包会保存定义函数时的自由变量的绑定，所以即使avg_wrapper已被调用过，定义作用域已经销毁，但还能访问到自由变量

print(avg(11))
print(avg(12))

10.0
10.5
11.0
10.0
10.5
11.0


In [13]:
# nonlocal声明

def make_avg():
  count = 0
  total = 0
  def avg(num):
    # 这里是count=count+1，会重新声明count，count就会变成局部变量，所以会报未声明count的错误
    # 之前没遇到这个错误，是因为[]是可变对象，而int，str是不可变对象
    # nonlocal count，total
    count += 1
    total += num
    return total/count
  
  return avg

avg = make_avg()
print(avg(10))


UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [ ]:
# 实现一个简单的装饰器
# 这样的实现不支持关键字参数，而且被装饰函数的__name__和__doc__属性也被修改成了clocked的名字和__doc__
import time
import functools

def clock(func):
  def clocked(*args):
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ','.join(repr(arg) for arg in args)
    print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
    return result
  return clocked

def new_clock(func):
  @functools.wraps(func)
  def clocked(*args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    elapsed = time.time() - t0
    name = func.__name__
    arg_lst = []
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
    arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
    return result
  return clocked

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

print(factorial(20))


[0.00000033s] factorial(1) -> 1
[0.00136658s] factorial(2) -> 2
[0.00138138s] factorial(3) -> 6
[0.00138783s] factorial(4) -> 24
[0.00139371s] factorial(5) -> 120
[0.00139883s] factorial(6) -> 720
[0.00140392s] factorial(7) -> 5040
[0.00141071s] factorial(8) -> 40320
[0.00141579s] factorial(9) -> 362880
[0.00142058s] factorial(10) -> 3628800
[0.00142583s] factorial(11) -> 39916800
[0.00143067s] factorial(12) -> 479001600
[0.00143633s] factorial(13) -> 6227020800
[0.00144313s] factorial(14) -> 87178291200
[0.00144812s] factorial(15) -> 1307674368000
[0.00145308s] factorial(16) -> 20922789888000
[0.00145796s] factorial(17) -> 355687428096000
[0.00146275s] factorial(18) -> 6402373705728000
[0.00146746s] factorial(19) -> 121645100408832000
[0.00147292s] factorial(20) -> 2432902008176640000
2432902008176640000
